In [1]:
import os
import glob
import pandas as pd
from transformers import LEDTokenizer, LEDForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch

2024-11-10 21:47:12.502409: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 21:47:12.519241: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731255432.542218   13052 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731255432.549057   13052 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 21:47:12.571810: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
torch.cuda.is_available()

True

In [3]:
# Clone the repository
!git clone https://github.com/rajdeep345/ECTSum.git

fatal: destination path 'ECTSum' already exists and is not an empty directory.


In [4]:
ects_path = "ECTSum/data/final/train/ects/"
summaries_path = "ECTSum/data/final/train/gt_summaries/"

# Read ects text files
ects_files = glob.glob(os.path.join(ects_path, "*.txt"))
ects_data = {}
for filepath in ects_files:
    with open(filepath, 'r', encoding='utf-8') as file:
        filename = os.path.basename(filepath)
        ects_data[filename] = file.read()

# Read gt_summaries text files
summaries_files = glob.glob(os.path.join(summaries_path, "*.txt"))
summaries_data = {}
for filepath in summaries_files:
    with open(filepath, 'r', encoding='utf-8') as file:
        filename = os.path.basename(filepath)
        summaries_data[filename] = file.read()

# ects_data and summaries_data now contain the contents of the files
print("Number of ECTS files loaded:", len(ects_data))
print("Number of Summaries files loaded:", len(summaries_data))

Number of ECTS files loaded: 1681
Number of Summaries files loaded: 1681


In [5]:
val_ects_path = "ECTSum/data/final/val/ects/"
val_summaries_path = "ECTSum/data/final/val/gt_summaries/"

# Read ects text files
val_ects_files = glob.glob(os.path.join(val_ects_path, "*.txt"))
val_ects_data = {}
for filepath in val_ects_files:
    with open(filepath, 'r', encoding='utf-8') as file:
        filename = os.path.basename(filepath)
        val_ects_data[filename] = file.read()

# Read gt_summaries text files
val_summaries_files = glob.glob(os.path.join(val_summaries_path, "*.txt"))
val_summaries_data = {}
for filepath in val_summaries_files:
    with open(filepath, 'r', encoding='utf-8') as file:
        filename = os.path.basename(filepath)
        val_summaries_data[filename] = file.read()

# ects_data and summaries_data now contain the contents of the files
print("Number of ECTS files loaded:", len(val_ects_data))
print("Number of Summaries files loaded:", len(val_summaries_data))

Number of ECTS files loaded: 249
Number of Summaries files loaded: 249


In [6]:
test_ects_path = "ECTSum/data/final/test/ects/"
test_summaries_path = "ECTSum/data/final/test/gt_summaries/"

# Read ects text files
test_ects_files = glob.glob(os.path.join(test_ects_path, "*.txt"))
test_ects_data = {}
for filepath in test_ects_files:
    with open(filepath, 'r', encoding='utf-8') as file:
        filename = os.path.basename(filepath)
        test_ects_data[filename] = file.read()

# Read gt_summaries text files
test_summaries_files = glob.glob(os.path.join(test_summaries_path, "*.txt"))
test_summaries_data = {}
for filepath in test_summaries_files:
    with open(filepath, 'r', encoding='utf-8') as file:
        filename = os.path.basename(filepath)
        test_summaries_data[filename] = file.read()

# test_ects_data and test_summaries_data now contain the contents of the files
print("Number of ECTS files loaded:", len(test_ects_data))
print("Number of Summaries files loaded:", len(test_summaries_data))


Number of ECTS files loaded: 495
Number of Summaries files loaded: 495


In [7]:
file_names = []
for file_name in ects_data.keys():
    file_names.append(file_name)
data = []
for file_name in file_names:
    data_dict = {'text':ects_data[file_name], 'summary': summaries_data[file_name]}
    data.append(data_dict)
train_df = pd.DataFrame(data)
train_df

,text,summary
0,"Before we begin our call today, I want to remi...",compname posts q2 adjusted loss per share $0.1...
1,"I'm Kelsey Duffy, Vice President of Investor R...",compname reports q3 earnings per share $1.66.\...
2,We have slides for our conference call.\nYou c...,compname says recent acquisitions to add fy sa...
3,Sales in the second quarter were $129.6 millio...,q2 earnings per share $0.03.\nq2 sales $129.6 ...
4,We begin with our Safe Harbor statement.\nYou ...,sees fy adjusted earnings per share $14.75 to ...
...,...,...
1676,Slide two contains our Safe Harbor statement.\...,compname reports q3 adjusted earnings per shar...
1677,We appreciate you're joining us today for Gart...,q1 revenue rose 8.4 percent to $1.1 billion.\n...
1678,"With us on the call today are Michael Kasbar, ...",compname reports q1 adjusted earnings per shar...
1679,"Yesterday after the close of market, we announ...",compname reports q1 earnings per share $0.17.\...


In [8]:
file_names_val = []
for file_name in val_ects_data.keys():
    file_names_val.append(file_name)
val_data = []
for file_name in file_names_val:
    val_data_dict = {'text':val_ects_data[file_name], 'summary': val_summaries_data[file_name]}
    val_data.append(val_data_dict)
val_df = pd.DataFrame(val_data)
val_df

,text,summary
0,"Well, we had an exciting quarter.\nI'm not sur...",compname reports q3 ffo per share $1.85 exclud...
1,Please note that this conference is being reco...,mdc holdings sees q4 2020 home deliveries betw...
2,"Before we get started, I want to let you know ...",q1 gaap earnings per share $5.52.\nsees fy adj...
3,"Carter's is making good progress, recovering f...",compname reports q3 earnings per share of $1.8...
4,dollargeneral.com under News & Events.\nWe als...,q4 earnings per share $2.62.\nq4 same store sa...
...,...,...
244,"Factors, to our Annual Report on Form 10-K for...",abbott sold more than 225 mln covid-19 tests g...
245,"If you do not yet have a copy, you can access ...",q4 adjusted ffo loss per share $0.16.17 hotel ...
246,"Kessel Stelling, chairman and chief executive ...",compname announces q3 earnings per share $0.56...
247,"In particular, the extent of the continued imp...",q2 non-gaap earnings per share $0.91.\nq2 gaap...


In [9]:
file_names_test = []
for file_name in test_ects_data.keys():
    file_names_test.append(file_name)

test_data = []
for file_name in file_names_test:
    test_data_dict = {'text': test_ects_data[file_name], 'summary': test_summaries_data[file_name]}
    test_data.append(test_data_dict)

test_df = pd.DataFrame(test_data)
test_df


,text,summary
0,"They are based on management's assumptions, wh...",compname reports q2 loss per share $0.17 from ...
1,In the third quarter Cullen/Frost earned $109....,q3 earnings per share $1.73.\nincreases quarte...
2,"With me on the call today are Seamus Grady, Ch...",compname reports q2 revenue $453.8 mln.\nsees ...
3,These factors are detailed in the company's fi...,qtrly optical communications sales grew 24% yo...
4,"2020 was an extraordinary year, and the consis...","compname reports earnings per share of $2.54, ..."
...,...,...
490,"Today, we will review the second quarter 2021 ...",q2 non-gaap earnings per share $3.54.\nq2 earn...
491,"Jim Hatfield, Chief Administrative Officer; Da...",compname reports q1 earnings per share of $0.3...
492,I will then review our segment performance and...,compname reports qtrly adjusted earnings per s...
493,These statements reflect the participants' exp...,q1 non-gaap earnings per share $0.79 from cont...


In [10]:
from transformers import T5Tokenizer

# Initialize the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Define max lengths
max_input_length = 4096
max_output_length = 256

# Preprocessing function
def preprocess_data(row):
    text = row['text']
    summary = row['summary']
    
    # Tokenize input text
    inputs = tokenizer(
        f"summarize: {text}",
        max_length=max_input_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    # Tokenize target summary
    targets = tokenizer(
        summary,
        max_length=max_output_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    return {
        "input_ids": inputs["input_ids"].squeeze(),
        "attention_mask": inputs["attention_mask"].squeeze(),
        "labels": targets["input_ids"].squeeze()
    }

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [11]:
# Preprocess the entire dataset
train_data = [preprocess_data(row) for _, row in train_df.iterrows()]
val_data = [preprocess_data(row) for _, row in val_df.iterrows()]
test_data = [preprocess_data(row) for _, row in test_df.iterrows()]

In [12]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [13]:
train_dataset

Dataset({
    features: ['text', 'summary'],
    num_rows: 1681
})

In [14]:
from transformers import T5ForConditionalGeneration, AdamW

# Initialize the T5 model and move it to the GPU if available
model = T5ForConditionalGeneration.from_pretrained('t5-small')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=3e-5)

/home/amitesh_sah/anaconda3/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
epochs = 5

In [16]:
from tqdm import tqdm

In [17]:
# Training loop with progress bar
model.train()
for epoch in range(epochs):
    total_loss = 0  # To accumulate the loss for averaging
    
    # Wrap the data iterator with tqdm to display a progress bar
    progress_bar = tqdm(train_data, desc=f"Epoch {epoch + 1}", leave=False)
    step = 0
    
    for data in progress_bar:
        input_ids = data["input_ids"].unsqueeze(0).to(device)
        attention_mask = data["attention_mask"].unsqueeze(0).to(device)
        labels = data["labels"].unsqueeze(0).to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()  # Accumulate the loss
        loss.backward()
        optimizer.step()
        
        step += 1
        
        # Update progress bar with the current loss
        progress_bar.set_postfix({"Loss": loss.item()})

    # Print average loss for the epoch
    avg_loss = total_loss / step
    print(f"Epoch {epoch + 1} completed, Average Loss: {avg_loss}")

Epoch 1:   0%|                                         | 0/1681 [00:00<?, ?it/s]Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1 completed, Average Loss: 1.0899445229253728


Epoch 2 completed, Average Loss: 0.658503212622527


Epoch 3 completed, Average Loss: 0.5881197213276707


Epoch 4 completed, Average Loss: 0.5466246971697644


Epoch 5 completed, Average Loss: 0.5184591745077934


In [18]:
import evaluate

In [19]:
# Load all necessary metrics
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")
meteor_metric = evaluate.load("meteor")
bert_score_metric = evaluate.load("bertscore")

model.to('cuda')

# Evaluation function to compute ROUGE, BLEU, BERTScore, and METEOR on test data
def evaluate_model_on_test(test_dataset):
    model.eval()  # Set model to evaluation mode
    predictions = []
    references = []
    
    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch in tqdm(test_dataset, desc="Evaluating", leave=False):
            # Convert input_ids and attention_mask to tensors
            input_ids = torch.tensor(batch["input_ids"]).unsqueeze(0).to('cuda')  # Add batch dimension
            attention_mask = torch.tensor(batch["attention_mask"]).unsqueeze(0).to('cuda')
            
            # Generate summary
            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150)
            pred_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            true_summary = tokenizer.decode(batch["labels"], skip_special_tokens=True)
            
            # Collect predictions and references
            predictions.append(pred_summary)
            references.append(true_summary)
    
    # Compute ROUGE scores
    rouge_scores = rouge_metric.compute(predictions=predictions, references=references, use_stemmer=True)
    
    # Compute BLEU scores (requires references as a list of lists)
    bleu_scores = bleu_metric.compute(predictions=predictions, references=[[ref] for ref in references])
    
    # Compute METEOR scores
    meteor_scores = meteor_metric.compute(predictions=predictions, references=references)
    
    # Compute BERTScore (useful for semantic similarity)
    bert_scores = bert_score_metric.compute(predictions=predictions, references=references, lang="en")
    bert_f1 = sum(bert_scores["f1"]) / len(bert_scores["f1"])  # Average F1 score from BERTScore
    
    # Return all metrics
    return {
        "rouge1": rouge_scores["rouge1"],
        "rouge2": rouge_scores["rouge2"],
        "rougeL": rouge_scores["rougeL"],
        "bleu": bleu_scores["bleu"],
        "meteor": meteor_scores["meteor"],
        "bertscore_f1": bert_f1  # BERTScore F1 (semantic similarity score)
    }

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/amitesh_sah/nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     /home/amitesh_sah/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/amitesh_sah/nltk_data...


In [20]:
# Run evaluation
test_metrics = evaluate_model_on_test(test_data)
print("Evaluation Results on Test dataset:")
print(test_metrics)

Evaluating:   0%|                                       | 0/495 [00:00<?, ?it/s]/tmp/ipykernel_13052/3850840905.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch["input_ids"]).unsqueeze(0).to('cuda')  # Add batch dimension
/tmp/ipykernel_13052/3850840905.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch["attention_mask"]).unsqueeze(0).to('cuda')


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation Results on Test dataset:
{'rouge1': 0.3562540031356058, 'rouge2': 0.23702649554799832, 'rougeL': 0.326090479415388, 'bleu': 0.20590181031343793, 'meteor': 0.3670692529604096, 'bertscore_f1': 0.8792363548519635}


In [21]:
# Save the model and tokenizer
model.save_pretrained("t5_finetuned_5epochs")
tokenizer.save_pretrained("t5_finetuned_5epochs")

('t5_finetuned_5epochs/tokenizer_config.json',
 't5_finetuned_5epochs/special_tokens_map.json',
 't5_finetuned_5epochs/spiece.model',
 't5_finetuned_5epochs/added_tokens.json')

In [22]:
def generate_summary(text):
    # Preprocess the input text
    inputs = tokenizer(
        f"summarize: {text}", 
        max_length=512, 
        padding="max_length", 
        truncation=True, 
        return_tensors="pt"
    )
    
    # Move tensors to the appropriate device
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    
    # Generate the summary (output IDs)
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150, num_beams=2, early_stopping=True)
    
    # Decode the generated summary IDs to text
    summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return summary

In [28]:
sample_text = """
Apple announced a new partnership with a leading tech firm, Google, to develop advanced AI solutions, in Quarter 4 of 2023. 
This collaboration aims to leverage the expertise of both companies in artificial intelligence and machine learning. 
The partnership is expected to accelerate innovation and improve product offerings in key markets. 
Both companies have expressed excitement about the potential of this joint venture to drive growth and create value for customers.
"""
summary = generate_summary(sample_text)
print("Generated Summary:", summary)
print(len(sample_text), '-->', len(summary))

Generated Summary: q4 partnership aims to leverage the expertise of both companies in artificial intelligence and machine learning.
475 --> 112


In [30]:
sample_text2 = """
Apple announced a new partnership with Google to develop advanced AI solutions, in Quarter 4 of 2023. 
Both companies have expressed excitement about the potential of this joint venture to drive growth and create value for customers.
The partnership is expected to accelerate innovation and improve product offerings in key markets. 
"""
summary2 = generate_summary(sample_text2)
print("Generated Summary:", summary2)
print(len(sample_text2), '-->', len(summary2))

Generated Summary: the partnership is expected to accelerate innovation and improve product offerings in key markets.
335 --> 98
